# VacationPy
---

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json 
import numpy as np

from config import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:\\Users\\juliu\\OneDrive\\Desktop\\penn-repositories\\python-api-challenge\\WeatherPy\\output_data\\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yurimaguas,-5.9000,-76.0833,23.88,92,100,1.37,PE,1707794527
1,1,ilulissat,69.2167,-51.1000,-24.99,58,74,1.54,GL,1707794527
2,2,fortaleza,-3.7227,-38.5247,29.01,83,40,3.60,BR,1707794401
3,3,georgetown,5.4112,100.3354,30.97,58,20,2.57,MY,1707794528
4,4,thunder bay,48.4001,-89.3168,-10.01,85,0,1.54,CA,1707794528


In [13]:
len(city_data_df)

565

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
# Configure the map plot

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 550,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# YOUR CODE HERE
import pandas as pd

# Step 1: Filter cities based on criteria
filtered_cities = city_data_df[(city_data_df['Humidity'] < 25) & (city_data_df['Max Temp'] < 25) & (city_data_df['Cloudiness'] <10)]

# Drop any rows with null values
filtered_cities_without_null = filtered_cities.dropna()

# Display sample data
filtered_cities_without_null

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
118,118,bardai,11.6500,7.2333,22.18,11,0,3.99,NG,1707794564
170,170,faya-largeau,17.9257,19.1043,22.74,12,0,3.90,TD,1707794575
258,258,tezu,27.9167,96.1667,22.47,23,7,0.61,IN,1707794594
262,262,biltine,14.5333,20.9167,21.37,11,1,3.70,TD,1707794595
264,264,villa regina,-39.1000,-67.0667,18.88,15,0,0.89,AR,1707794595
297,297,gumel,12.6279,9.3912,20.48,11,0,4.05,NG,1707794602
399,399,gashua,12.8710,11.0482,20.54,12,0,4.30,NG,1707794625
492,492,toungo,8.1167,12.0500,23.46,20,0,0.62,NG,1707794657
537,537,nguigmi,14.2495,13.1092,21.17,16,8,4.94,NE,1707794667


In [16]:
len(filtered_cities_without_null)

9

### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns_to_keep = ['City', 'Country', 'Lat', 'Lng', 'Humidity']

hotel_df = filtered_cities_without_null[columns_to_keep]

hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

C:\Users\juliu\AppData\Local\Temp\ipykernel_66760\2030370145.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
118,bardai,NG,11.6500,7.2333,11,
170,faya-largeau,TD,17.9257,19.1043,12,
258,tezu,IN,27.9167,96.1667,23,
262,biltine,TD,14.5333,20.9167,11,
264,villa regina,AR,-39.1000,-67.0667,15,
297,gumel,NG,12.6279,9.3912,11,
399,gashua,NG,12.8710,11.0482,12,
492,toungo,NG,8.1167,12.0500,20,
537,nguigmi,NE,14.2495,13.1092,16,


In [18]:
len(hotel_df)

9

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":'categories',
    "conditions":'conditions',
    "limit":'limit',
    "filter":'filters',
    "bias":'bias',
    "apiKey":'geoapify_key'  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    city = row['City']
    country = row['Country']
    latitude = row['Lat']
    longitude = row ['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    params = {
    'city': city,
    'country': country,
    'max_distance': 10000
    }
    
    # Convert the API response to JSON format
    places_data = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = places_data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bardai - nearest hotel: No hotel found
faya-largeau - nearest hotel: No hotel found
tezu - nearest hotel: No hotel found
biltine - nearest hotel: No hotel found
villa regina - nearest hotel: No hotel found
gumel - nearest hotel: No hotel found
gashua - nearest hotel: No hotel found
toungo - nearest hotel: No hotel found
nguigmi - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
118,bardai,NG,11.6500,7.2333,11,No hotel found
170,faya-largeau,TD,17.9257,19.1043,12,No hotel found
258,tezu,IN,27.9167,96.1667,23,No hotel found
262,biltine,TD,14.5333,20.9167,11,No hotel found
264,villa regina,AR,-39.1000,-67.0667,15,No hotel found
297,gumel,NG,12.6279,9.3912,11,No hotel found
399,gashua,NG,12.8710,11.0482,12,No hotel found
492,toungo,NG,8.1167,12.0500,20,No hotel found
537,nguigmi,NE,14.2495,13.1092,16,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 800,
    scale = 1.0,
    size = 100,
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)